In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [110]:
graph, random_solution, total_edge_length = dg.read_scientific_instance('data\\nug_23_5.txt')
num_nodes = len(graph)

In [111]:
sols = []
values = []
iters = []
times = []

<span style="font-size: larger;">Classic local search</span>

In [112]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    index1, index2 = random.sample(range(len(solution)), 2)
    
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [113]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [114]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    #we will be using this in a modified local search when regular methods get stuck in hopes of finding
    #a better solution close to the current one
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    if k == -1:
        return sorted(new_solution)

    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [115]:
def make_change_scramble(graph, solution):
    #another change function, as the name suggest, this one will scramble a section of the solution
    #this wouldn't be the method we use if we only tested one version, but since there are plenty of change functions and combinations,
    #this one might prove most useful in larger dimension
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [116]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [117]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[17, 6, 22, 10, 20, 1, 18, 13, 3, 14, 15, 8, 5, 9, 21, 4, 19, 0, 11, 2, 16, 7, 12] 1605 476


In [118]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 5, 12, 20, 19, 2, 13, 1, 18, 15, 7, 8, 14, 10, 9, 3, 4, 22, 21, 16, 0, 11, 17] 1601 347


In [119]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[12, 0, 1, 19, 18, 6, 7, 13, 20, 14, 2, 8, 9, 21, 15, 4, 3, 10, 22, 16, 17, 11, 5] 1604 454


In [120]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,23,make_change_swap,1605,0.10
1,23,make_change_inverse,1601,0.11
2,23,make_change_scramble,1604,0.13


In [121]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 1601 0.11 347


,Dim,Method,Value,Time,Best_Iter
0,23,make_change_inverse,1601,0.11,347


In [122]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

#check if first write to the specified row
if not row_to_update.empty:
    #update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_time'] = times[best_i]

    #save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search': value, 'local_search_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [123]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [1603.33, 0.11, 425.67]


,Dim,Value,Time,Best_Iter
0,23,1603.33,0.11,425.67


In [124]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search'] = average[0]
    df.loc[row_to_update.index, 'local_search_time'] = average[1]


    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search': average[0], 'local_search_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Local search with next permutation help</span>

In [125]:
sols = []
values = []
iters = []
times = []

def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = dg.calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i

In [126]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[11, 0, 12, 7, 19, 21, 2, 16, 9, 4, 14, 8, 15, 13, 18, 3, 10, 22, 20, 1, 6, 5, 17] 1584 434


In [127]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 12, 5, 2, 20, 1, 19, 13, 9, 7, 15, 8, 14, 10, 18, 3, 4, 16, 21, 22, 0, 11, 17] 1600 401


In [128]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[11, 12, 0, 19, 7, 16, 4, 21, 2, 14, 9, 15, 13, 8, 1, 3, 22, 18, 10, 20, 6, 17, 5] 1585 442


In [129]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__ + '_with_permutation'
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i    

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,23,make_change_swap_with_permutation,1584,0.82
1,23,make_change_inverse_with_permutation,1600,0.66
2,23,make_change_scramble_with_permutation,1585,0.70


In [130]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__ + '_with_permutation', 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_swap 1584 0.82 434


,Dim,Method,Value,Time,Best_Iter
0,23,make_change_swap_with_permutation,1584,0.82,434


In [131]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search_perm'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_perm_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search_perm': value, 'local_search_perm_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [132]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [1589.67, 0.73, 425.67]


,Dim,Value,Time,Best_Iter
0,23,1589.67,0.73,425.67


In [133]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    df.loc[row_to_update.index, 'local_search_perm'] = average[0]
    df.loc[row_to_update.index, 'local_search_perm_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search_perm': average[0], 'local_search_perm_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Simulated annealing</span>

In [134]:
sols = []
values = []
iters = []
times = []

def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [135]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 5, 1, 18, 17, 20, 10, 3, 22, 13, 15, 8, 9, 14, 16, 21, 4, 7, 19, 0, 2, 12, 11] 1585 428


In [136]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[11, 0, 12, 2, 21, 7, 19, 16, 4, 9, 14, 15, 8, 3, 13, 1, 20, 10, 22, 18, 6, 5, 17] 1582 500


In [137]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 5, 18, 17, 22, 10, 20, 1, 3, 13, 14, 19, 15, 9, 8, 21, 4, 2, 16, 7, 0, 11, 12] 1595 236


In [138]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': 'simulated_annealing' + ', '  + method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,23,"simulated_annealing, make_change_swap",1585,0.12
1,23,"simulated_annealing, make_change_inverse",1582,0.12
2,23,"simulated_annealing, make_change_scramble",1595,0.13


In [139]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  'simulated_annealing' + ', '  + solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 1582 0.12 500


,Dim,Method,Value,Time,Best_Iter
0,23,"simulated_annealing, make_change_inverse",1582,0.12,500


In [140]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = values[best_i]
    df.loc[row_to_update.index, 'sim_annealing_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'sim_annealing': value, 'sim_annealing_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [141]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [1587.33, 0.12, 388.0]


,Dim,Value,Time,Best_Iter
0,23,1587.33,0.12,388.0


In [142]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = average[0]
    df.loc[row_to_update.index, 'sim_annealing_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'sim_annealing_perm': average[0], 'sim_annealing_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Variable neigborhood search</span>

In [143]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [144]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [145]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            #diversification
            new_solution = shaking_func(graph, solution, k)   

            new_value = dg.calculate_total_edge_length(graph, new_solution)

            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)

            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i

                value = new_value

                solution = deepcopy(new_solution)
                
    return solution, value, best_i

In [158]:
#too many combos for VNS, we will iterrate
#this soulution SHOULD NOT be used regularly, as the cell execution time will be MUCH longer than the sum of it's parts for some reason (notebook problem?)
#for example, for a certain n, if we split this into seperate cells the sum of exec time would be 3mins, this way it's around 5-6mins
#hopefully this doesn't seem to affect individual duration calculations...
n = len(graph)

oc = False

shakings = [shaking_swap, shaking_inverse]
changes_funcs = [make_change_swap, make_change_inverse, make_change_scramble]
search_funcs = [local_search, local_search_with_permutation]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

k_min = int(0.1 * num_nodes)
k_max = int(0.3 * num_nodes)
for shake in shakings:
    for change in changes_funcs:
        for search in search_funcs:
            if shake.__name__ == 'shaking_inverse':
                k_min = int(0.3 * num_nodes)
                k_max = int(0.6 * num_nodes)

            start = time.time()    
            sol, value, iter = vns(graph, 
                                   random_solution, 
                                   total_edge_length, 
                                   500, 
                                   change_func=change, 
                                   shaking_func=shake, 
                                   local_search_func = search, 
                                   k_min=k_min,
                                   k_max=k_max, 
                                   move_prob=0.4)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(shake.__name__, change.__name__, search.__name__, ' : ', sol, value, iter, '\n')
            
            k_min = int(0.1 * num_nodes)
            k_max = int(0.3 * num_nodes)

            sols.append(sol)
            values.append(value)
            iters.append(iter)
            times.append(duration)

            methods.append(str(shake.__name__ + ', ' + change.__name__ + ', ' + search.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(value), 'Time': duration})

            if value == best:
                if duration <= times[best_i]:    
                    best = value
                    best_i = number_of_combinations

            if value < best:
                best = value
                best_i = number_of_combinations            

            average[0] += value
            average[1] += duration

            number_of_combinations += 1    


shaking_swap make_change_swap local_search  :  [11, 0, 12, 21, 16, 4, 2, 7, 14, 19, 9, 15, 8, 13, 3, 10, 18, 20, 22, 1, 6, 5, 17] 1584 408 

shaking_swap make_change_swap local_search_with_permutation  :  [12, 11, 0, 21, 7, 2, 4, 19, 16, 14, 9, 15, 8, 3, 13, 18, 10, 22, 1, 5, 20, 17, 6] 1583 285 

shaking_swap make_change_inverse local_search  :  [6, 5, 17, 1, 18, 20, 10, 22, 15, 3, 13, 8, 14, 4, 9, 21, 16, 7, 19, 0, 2, 11, 12] 1586 321 

shaking_swap make_change_inverse local_search_with_permutation  :  [12, 0, 19, 16, 4, 11, 7, 2, 21, 8, 14, 9, 3, 15, 13, 22, 1, 18, 20, 6, 10, 5, 17] 1588 360 

shaking_swap make_change_scramble local_search  :  [0, 12, 11, 19, 7, 21, 2, 4, 16, 14, 9, 8, 15, 13, 3, 20, 18, 1, 10, 6, 17, 22, 5] 1583 496 

shaking_swap make_change_scramble local_search_with_permutation  :  [0, 6, 12, 1, 19, 2, 7, 18, 13, 14, 8, 15, 20, 9, 21, 22, 3, 16, 10, 17, 4, 5, 11] 1606 436 

shaking_inverse make_change_swap local_search  :  [0, 6, 12, 19, 2, 1, 7, 13, 14, 9, 20, 

In [159]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/vns.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/vns.csv'), index=False)

,Dim,Method,Value,Time
0,23,"shaking_swap, make_change_swap, local_search",1584,4.44
1,23,"shaking_swap, make_change_swap, local_search_w...",1583,12.65
2,23,"shaking_swap, make_change_inverse, local_search",1586,4.82
3,23,"shaking_swap, make_change_inverse, local_searc...",1588,12.70
4,23,"shaking_swap, make_change_scramble, local_search",1583,5.35
5,23,"shaking_swap, make_change_scramble, local_sear...",1606,13.44
6,23,"shaking_inverse, make_change_swap, local_search",1601,8.29
7,23,"shaking_inverse, make_change_swap, local_searc...",1588,22.58
8,23,"shaking_inverse, make_change_inverse, local_se...",1603,8.32
9,23,"shaking_inverse, make_change_inverse, local_se...",1600,22.49


In [160]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: shaking_swap, make_change_scramble, local_search 1583 5.35


,Dim,Method,Value,Time
0,23,"shaking_swap, make_change_scramble, local_search",1583,5.35


In [161]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:
    
    df.loc[row_to_update.index, 'vns'] = values[best_i]
    df.loc[row_to_update.index, 'vns_time'] = times[best_i]

    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'vns': value, 'vns_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [162]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [1592.5, 12.34]


,Dim,Value,Time
0,23,1592.5,12.34


In [163]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

if not row_to_update.empty:

    df.loc[row_to_update.index, 'vns'] = average[0]
    df.loc[row_to_update.index, 'vns_time'] = average[1]

    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'vns_perm': average[0], 'vns_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)